In [29]:
'''
=================================================
Milestone 1

Nama  : Hazna Dhifa Putri Ardeva
Batch : CODA-RMT-014

Milestones ini dibuat untuk mengerjakan project data pipeline/ETL 
=================================================
'''

'\n=================================================\nMilestone 1\n\nNama  : Hazna Dhifa Putri Ardeva\nBatch : CODA-RMT-014\n\nMilestones ini dibuat untuk mengerjakan project data pipeline/ETL \n=================================================\n'

# Extract

Melakukan pengambilan data dari website booking.com untuk mendapatkan data informasi 'hotel dan penginapan di Bali' dalam rentang waktu dari 1 - 4 Januari 2026.
- Link web : https://www.booking.com/searchresults.html?aid=304142&label=gen173nr-10CAEoggI46AdIM1gEaGiIAQGYATO4ARfIAQzYAQPoAQH4AQGIAgGoAgG4ApSzs8oGwAIB0gIkOTAyYTAzM2EtMzcyNC00M2EzLTk3ZmEtNDlkNDA0NGYzYjhk2AIB4AIB&checkin=2026-01-01&checkout=2026-01-04&dest_id=835&dest_type=region&order=popularity&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&previous_search_id=428532f5-742a-4caa-9e0b-d3e66592a53c


In [1]:
# Install dan import library yang dibutuhkan
!pip install pandas bs4 selenium

# Data Manipulation
import pandas as pd

# Webscraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Web Scraping

# Input link website yang akan di scraping
driver = webdriver.Chrome()
url = 'https://www.booking.com/searchresults.html?aid=304142&label=gen173nr-10CAEoggI46AdIM1gEaGiIAQGYATO4ARfIAQzYAQPoAQH4AQGIAgGoAgG4ApSzs8oGwAIB0gIkOTAyYTAzM2EtMzcyNC00M2EzLTk3ZmEtNDlkNDA0NGYzYjhk2AIB4AIB&checkin=2026-01-01&checkout=2026-01-04&dest_id=835&dest_type=region&order=popularity&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&previous_search_id=428532f5-742a-4caa-9e0b-d3e66592a53c'
driver.get(url)

# Membaca seluruh data pada link website
'''
    Link website yang akan dilakukan scraping data menggunakan mekanisme "infinite scroll", 
    yang artinya data hotel dimuat secara bertahap. Untuk mengambil data dengan sistem "infinite scroll" ini,
    dilakukan looping terhdap web dengan cara:
    - mengukur panjang halaman web hingga data terakhir
    - memuat data baru, membaca data baru, dan mengukur panjang halaman web setelah data baru di muat 
    - membandingkan panjang halaman web setelah data baru dimuat dengan panjang halaman web hingga data terakhir
    - Jika panjang halaman web setelah data baru dimuat sama dengan hasil pengukuran panjang halaman web hingga data terakhir,
      maka loop akan dihentikan, karena data sudah terambil seluruhnya  
'''
last_height = driver.execute_script("return document.body.scrollHeight") 

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # mengukur panjang halaman web hingga data terakhir
    time.sleep(2) # waktu untuk load

    new_height = driver.execute_script("return document.body.scrollHeight") # mengukur panjang halaman web setelah data baru di muat 
    if new_height == last_height:
        break 
    last_height = new_height

# Mengambil html
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

Setelah semua data pada web dapat terbaca, selanjutnya dilakukan pengambilan informasi data yang diperlukan. Pada case ini, informasi yang diperlukan yaitu:
- Nama Hotel 
- Lokasi
- Rating
- Spesial Promo
- Harga

In [3]:
# Membuat list kosong untuk kolom yang akan diperlukan 
nama_hotel_clean = []
lokasi_clean =[]
rating_clean= []
spesial_promo_clean = []
harga_clean = []

In [4]:
boxes = soup.find_all('div',{'data-testid':'property-card-container'})

# Mengisi list kosong dengan data 
for box in boxes:
    try:
        nama_hotel = box.find('div',{'data-testid':'title'})
        nama_hotel_clean.append(nama_hotel.get_text())
    except:
        nama_hotel_clean.append(None)
    
    try:
        lokasi = box.find('span',{'data-testid':'address'})
        lokasi_clean.append(lokasi.get_text())
    except:
        lokasi_clean.append(None)

    try:
        rating = box.find('div',{'data-testid':'review-score'})
        rating_clean.append(rating.get_text())
    except:
        rating_clean.append(None)

    try:
        spesial_promo = box.find('span',{'data-testid':'property-card-deal'})
        spesial_promo_clean.append(spesial_promo.get_text())
    except:
        spesial_promo_clean.append(None)

    try:
        harga = box.find('span',{'data-testid':'price-and-discounted-price'})
        harga_clean.append(harga.get_text())
    except:
        harga_clean.append(None)

driver.quit() 

In [5]:
# Melihat dan memastikan panjang masing-masing data 
print(len(nama_hotel_clean)) 
print(len(lokasi_clean))
print(len(rating_clean))
print(len(spesial_promo_clean))
print(len(harga_clean)) 


75
75
75
75
75


In [ ]:
# Menginput data hasil scrap ke dalam bentuk dataframe
df = pd.DataFrame()
df['nama_hotel'] = nama_hotel_clean
df['lokasi'] = lokasi_clean
df['rating'] = rating_clean
df['spesial_promo'] = spesial_promo_clean
df['harga'] = harga_clean
df

,nama_hotel,lokasi,rating,spesial_promo,harga
0,Aksmi Suites & Villas,Bedahulu,Scored 9.0 9.0Wonderful 3 reviews,Limited-time Deal,"Rp 3,037,500"
1,Villa Mahayoni,Bedahulu,Scored 9.8 9.8Exceptional 9 reviews,Early 2026 Deal,"Rp 2,368,800"
2,Kelingking Parama Resort,batumadeg,Scored 9.4 9.4Wonderful 462 reviews,None,"Rp 3,427,686"
3,Kelingking Puri Suci Villa,batumadeg,Scored 9.1 9.1Wonderful 194 reviews,Late Escape Deal,"Rp 2,347,650"
4,Sky Star Ocean View Uluwatu,"Ungasan, Uluwatu",Scored 10 10Exceptional 3 reviews,None,"Rp 9,355,135"
...,...,...,...,...,...
69,Villa Markisa Pakisan,Sawan,Scored 9.7 9.7Exceptional 194 reviews,None,"Rp 1,252,555"
70,LA Beach Nusa Penida,Nusa Penida,Scored 8.6 8.6Excellent 138 reviews,None,"Rp 1,701,000"
71,Nido Boutique Cottage Canggu,"Babakan, Tumbak Bayuh (Babakan)",Scored 10 10Exceptional 5 reviews,None,"Rp 3,780,000"
72,Graha Shanti Guest House,Pulau Serangan,Scored 9.0 9.0Wonderful 132 reviews,None,"Rp 1,227,273"


In [ ]:
# Menyimpan dataframe ke dalam file csv
df.to_csv('hotel_bali.csv')

# Transform
---
Pada tahap ini, dilakukan eksplorasi data sederhana dari hasil web scraping menggunakan library Pandas. 
Exploratory Data Analyst yang dilakukan pada tahap ini yaitu:
- Kesesuaian kolom dengan isi kolom
- Kesesuaian tipe data tiap kolom

Hasil proses EDA ini nantinya disimpan dalam bentuk csv


### Data Understanding
---
Dataset merupakan data hasil web scraping booking.com, dimana dataset berisi informasi booking hotel di Bali mulai tanggal 1 - 4 Januari 2026. Dataset harus memiliki informasi column sebagai berikut:

| Column | Description
| :--- | :---
| hotel_id | ID hotel (int)
| nama_hotel | nama hotel (object)
| lokasi | lokasi tempat hotel (object)
| rating | rating score hotel (float)
| kategori_rating | kategori rating hotel (object)
| spesial_promo | promo yang disediakan hotel (object)
| harga | harga hotel (float)

### Exploratory Data Analyst


In [3]:
# Upload dataset
df = pd.read_csv('F:\DATA\DHIFA\HACTIV8\hotel_bali.csv')
df.head()

,Unnamed: 0,nama_hotel,lokasi,rating,spesial_promo,harga
0,0,Aksmi Suites & Villas,Bedahulu,Scored 9.0 9.0Wonderful 3 reviews,Limited-time Deal,"Rp 3,037,500"
1,1,Villa Mahayoni,Bedahulu,Scored 9.8 9.8Exceptional 9 reviews,Early 2026 Deal,"Rp 2,368,800"
2,2,Kelingking Parama Resort,batumadeg,Scored 9.4 9.4Wonderful 462 reviews,NaN,"Rp 3,427,686"
3,3,Kelingking Puri Suci Villa,batumadeg,Scored 9.1 9.1Wonderful 194 reviews,Late Escape Deal,"Rp 2,347,650"
4,4,Sky Star Ocean View Uluwatu,"Ungasan, Uluwatu",Scored 10 10Exceptional 3 reviews,NaN,"Rp 9,355,135"


In [4]:
# Melihat informasi data (tipe data, missing value, jumlah column)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     74 non-null     int64 
 1   nama_hotel     74 non-null     object
 2   lokasi         74 non-null     object
 3   rating         73 non-null     object
 4   spesial_promo  21 non-null     object
 5   harga          74 non-null     object
dtypes: int64(1), object(5)
memory usage: 3.6+ KB


Berdasarkan informasi dan hasil check isi data di atas, dapat dilihat bahwa beberapa data masih belum sesuai dengan data description, yaitu:
- Terdapat tipe data yang tidak sesuai dengan data description
- Terdapat kolom yang kurang
- Terdapat nama dan isi kolom yang belum sesuai

sehingga perlu dilakukan data cleaning lebih lanjut

#### Data Cleaning

In [5]:
df

,Unnamed: 0,nama_hotel,lokasi,rating,spesial_promo,harga
0,0,Aksmi Suites & Villas,Bedahulu,Scored 9.0 9.0Wonderful 3 reviews,Limited-time Deal,"Rp 3,037,500"
1,1,Villa Mahayoni,Bedahulu,Scored 9.8 9.8Exceptional 9 reviews,Early 2026 Deal,"Rp 2,368,800"
2,2,Kelingking Parama Resort,batumadeg,Scored 9.4 9.4Wonderful 462 reviews,NaN,"Rp 3,427,686"
3,3,Kelingking Puri Suci Villa,batumadeg,Scored 9.1 9.1Wonderful 194 reviews,Late Escape Deal,"Rp 2,347,650"
4,4,Sky Star Ocean View Uluwatu,"Ungasan, Uluwatu",Scored 10 10Exceptional 3 reviews,NaN,"Rp 9,355,135"
...,...,...,...,...,...,...
69,69,Villa Markisa Pakisan,Sawan,Scored 9.7 9.7Exceptional 194 reviews,NaN,"Rp 1,252,555"
70,70,LA Beach Nusa Penida,Nusa Penida,Scored 8.6 8.6Excellent 138 reviews,NaN,"Rp 1,701,000"
71,71,Nido Boutique Cottage Canggu,"Babakan, Tumbak Bayuh (Babakan)",Scored 10 10Exceptional 5 reviews,NaN,"Rp 3,780,000"
72,72,Graha Shanti Guest House,Pulau Serangan,Scored 9.0 9.0Wonderful 132 reviews,NaN,"Rp 1,227,273"


Data cleaning yang dilakukan yaitu memisahkan value pada kolom 'rating' menjadi 2 kolom yang berisikan skor rating (float) dan kategori rating (str)

In [6]:
# Membuat dua kolom kosong
df['skor_rating'] = None
df['kategori_rating'] = None 

In [7]:
'''
    Pengisian kolom baru dilakukan dengan looping. Looping ini dilakukan dengan mengambil setiap row di kolom rating,
    kemudian melakukan split untuk memisahkan masing-masing isi kata dalam kolom rating, dan mengambil angka rating 
    (9.0, 8.1, ...) untuk diinput dalam kolom baru 'skor_rating' dan mengambil kategori rating (Wonderful, Escellent, ...)
    untuk diinput dalam kolom baru 'kategori_rating'   
'''

for i in range(len(df)):
    rating_column = df.loc[i, 'rating'] # Mengambil tiap row/baris di kolom rating
    
    # Mengisi kolom baru (skor_rating dan kategori_rating) dengan None jika value di kolom rating isinya nan
    if pd.isna(rating_column): 
        df.loc[i, 'skor_rating'] = None
        df.loc[i, 'kategori_rating'] = None
        continue
    
    # Melakukan split teks pada kolom rating
    split_teks = rating_column.split()
    
    # Membuat variable kosong 
    angka_rating = None
    kategori = None
    
    # Mengambil teks berupa angka pada kolom rating dan mengubah nya dalam bentuk float, kemudian disimpoan dalam variable angka_rating 
    for angka_skor in split_teks:
        if '.' in angka_skor:
            try:
                angka_rating = float(angka_skor)
                break
            except:
                pass
    
        else:
            try:
                angka = int(angka_skor)
                if 0 <= angka <= 10:  
                    angka_rating = float(angka)
                    break
            except:
                pass
    
    # Mengambil teks berupa kata (wonderful, exceptional, excellent, dan very good) untuk diinput dalam variable kategori
    kata_kategori = ['Wonderful', 'Exceptional', 'Excellent', 'Very Good']
    for kata in kata_kategori:
        if kata in rating_column:
            kategori = kata
            break
    
    # Input value ke dalam kolom baru
    df.loc[i, 'skor_rating'] = angka_rating
    df.loc[i, 'kategori_rating'] = kategori 

In [8]:
# Check hasil input kolom baru 
df

,Unnamed: 0,nama_hotel,lokasi,rating,spesial_promo,harga,skor_rating,kategori_rating
0,0,Aksmi Suites & Villas,Bedahulu,Scored 9.0 9.0Wonderful 3 reviews,Limited-time Deal,"Rp 3,037,500",9.0,Wonderful
1,1,Villa Mahayoni,Bedahulu,Scored 9.8 9.8Exceptional 9 reviews,Early 2026 Deal,"Rp 2,368,800",9.8,Exceptional
2,2,Kelingking Parama Resort,batumadeg,Scored 9.4 9.4Wonderful 462 reviews,NaN,"Rp 3,427,686",9.4,Wonderful
3,3,Kelingking Puri Suci Villa,batumadeg,Scored 9.1 9.1Wonderful 194 reviews,Late Escape Deal,"Rp 2,347,650",9.1,Wonderful
4,4,Sky Star Ocean View Uluwatu,"Ungasan, Uluwatu",Scored 10 10Exceptional 3 reviews,NaN,"Rp 9,355,135",10.0,Exceptional
...,...,...,...,...,...,...,...,...
69,69,Villa Markisa Pakisan,Sawan,Scored 9.7 9.7Exceptional 194 reviews,NaN,"Rp 1,252,555",9.7,Exceptional
70,70,LA Beach Nusa Penida,Nusa Penida,Scored 8.6 8.6Excellent 138 reviews,NaN,"Rp 1,701,000",8.6,Excellent
71,71,Nido Boutique Cottage Canggu,"Babakan, Tumbak Bayuh (Babakan)",Scored 10 10Exceptional 5 reviews,NaN,"Rp 3,780,000",10.0,Exceptional
72,72,Graha Shanti Guest House,Pulau Serangan,Scored 9.0 9.0Wonderful 132 reviews,NaN,"Rp 1,227,273",9.0,Wonderful


In [9]:
# Mengatur ulang kolom di dataframe dan mengubah nama kolom
df = df[['Unnamed: 0', 'nama_hotel', 'lokasi', 'skor_rating', 'kategori_rating', 'spesial_promo', 'harga']]

df = df.rename(columns = {
    'Unnamed: 0': 'hotel_id',
    'skor_rating': 'rating'
})

df

,hotel_id,nama_hotel,lokasi,rating,kategori_rating,spesial_promo,harga
0,0,Aksmi Suites & Villas,Bedahulu,9.0,Wonderful,Limited-time Deal,"Rp 3,037,500"
1,1,Villa Mahayoni,Bedahulu,9.8,Exceptional,Early 2026 Deal,"Rp 2,368,800"
2,2,Kelingking Parama Resort,batumadeg,9.4,Wonderful,NaN,"Rp 3,427,686"
3,3,Kelingking Puri Suci Villa,batumadeg,9.1,Wonderful,Late Escape Deal,"Rp 2,347,650"
4,4,Sky Star Ocean View Uluwatu,"Ungasan, Uluwatu",10.0,Exceptional,NaN,"Rp 9,355,135"
...,...,...,...,...,...,...,...
69,69,Villa Markisa Pakisan,Sawan,9.7,Exceptional,NaN,"Rp 1,252,555"
70,70,LA Beach Nusa Penida,Nusa Penida,8.6,Excellent,NaN,"Rp 1,701,000"
71,71,Nido Boutique Cottage Canggu,"Babakan, Tumbak Bayuh (Babakan)",10.0,Exceptional,NaN,"Rp 3,780,000"
72,72,Graha Shanti Guest House,Pulau Serangan,9.0,Wonderful,NaN,"Rp 1,227,273"


In [10]:
# Ubah hotel_id mulai dari 1 
df['hotel_id'] = range(1, len(df) + 1)
df

,hotel_id,nama_hotel,lokasi,rating,kategori_rating,spesial_promo,harga
0,1,Aksmi Suites & Villas,Bedahulu,9.0,Wonderful,Limited-time Deal,"Rp 3,037,500"
1,2,Villa Mahayoni,Bedahulu,9.8,Exceptional,Early 2026 Deal,"Rp 2,368,800"
2,3,Kelingking Parama Resort,batumadeg,9.4,Wonderful,NaN,"Rp 3,427,686"
3,4,Kelingking Puri Suci Villa,batumadeg,9.1,Wonderful,Late Escape Deal,"Rp 2,347,650"
4,5,Sky Star Ocean View Uluwatu,"Ungasan, Uluwatu",10.0,Exceptional,NaN,"Rp 9,355,135"
...,...,...,...,...,...,...,...
69,70,Villa Markisa Pakisan,Sawan,9.7,Exceptional,NaN,"Rp 1,252,555"
70,71,LA Beach Nusa Penida,Nusa Penida,8.6,Excellent,NaN,"Rp 1,701,000"
71,72,Nido Boutique Cottage Canggu,"Babakan, Tumbak Bayuh (Babakan)",10.0,Exceptional,NaN,"Rp 3,780,000"
72,73,Graha Shanti Guest House,Pulau Serangan,9.0,Wonderful,NaN,"Rp 1,227,273"


In [11]:
# Check informasi data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   hotel_id         74 non-null     int64 
 1   nama_hotel       74 non-null     object
 2   lokasi           74 non-null     object
 3   rating           73 non-null     object
 4   kategori_rating  73 non-null     object
 5   spesial_promo    21 non-null     object
 6   harga            74 non-null     object
dtypes: int64(1), object(6)
memory usage: 4.2+ KB


Nama, isi kolom, dan jumlah kolom sudah sesuai, tahap cleaning data selanjutnya yaitu mengubah tipe data untuk kolom:
- rating dari object menjadi float
- harga dari object menjadi integer

In [12]:
'''
    Looping di bawah ini dilakukan untuk menghapus tanda koma, spasi, dan kata 'Rp' pada kolom 'harga' 
    untuk dapat mengubah tipe data 'harga' menjadi integer 
'''

for i in df.index:  
    harga_asli = str(df.loc[i, 'harga']) # Mengambil nilai harga tiap baris 
    
    # Menghapus tanda koma, spasi, dan 'Rp' dengan cara mengubahnya menjadi string kosong
    harga_bersih = harga_asli.replace('Rp', '')
    harga_bersih = harga_bersih.replace(' ', '')
    harga_bersih = harga_bersih.replace(',', '')
        
    # Menyimpan kembali hasil ke dalam dataframe
    harga_angka = harga_bersih
    df.loc[i, 'harga'] = harga_angka

In [13]:
# Mengubah tipe data
df['harga'] = df['harga'].astype(int)
df['rating'] = df['rating'].astype(float)

In [14]:
# Mengecheck hasil perubahan tipe data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   hotel_id         74 non-null     int64  
 1   nama_hotel       74 non-null     object 
 2   lokasi           74 non-null     object 
 3   rating           73 non-null     float64
 4   kategori_rating  73 non-null     object 
 5   spesial_promo    21 non-null     object 
 6   harga            74 non-null     int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 4.2+ KB


Berdasarkan informasi di atas, seluruh kolom sudah sesuai dengan data description

# Result

In [15]:
df

,hotel_id,nama_hotel,lokasi,rating,kategori_rating,spesial_promo,harga
0,1,Aksmi Suites & Villas,Bedahulu,9.0,Wonderful,Limited-time Deal,3037500
1,2,Villa Mahayoni,Bedahulu,9.8,Exceptional,Early 2026 Deal,2368800
2,3,Kelingking Parama Resort,batumadeg,9.4,Wonderful,NaN,3427686
3,4,Kelingking Puri Suci Villa,batumadeg,9.1,Wonderful,Late Escape Deal,2347650
4,5,Sky Star Ocean View Uluwatu,"Ungasan, Uluwatu",10.0,Exceptional,NaN,9355135
...,...,...,...,...,...,...,...
69,70,Villa Markisa Pakisan,Sawan,9.7,Exceptional,NaN,1252555
70,71,LA Beach Nusa Penida,Nusa Penida,8.6,Excellent,NaN,1701000
71,72,Nido Boutique Cottage Canggu,"Babakan, Tumbak Bayuh (Babakan)",10.0,Exceptional,NaN,3780000
72,73,Graha Shanti Guest House,Pulau Serangan,9.0,Wonderful,NaN,1227273


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   hotel_id         74 non-null     int64  
 1   nama_hotel       74 non-null     object 
 2   lokasi           74 non-null     object 
 3   rating           73 non-null     float64
 4   kategori_rating  73 non-null     object 
 5   spesial_promo    21 non-null     object 
 6   harga            74 non-null     int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 4.2+ KB


In [17]:
df.describe()

,hotel_id,rating,harga
count,74.000000,73.000000,7.400000e+01
mean,37.500000,9.057534,3.653523e+06
std,21.505813,0.537979,3.519978e+06
min,1.000000,8.100000,1.693990e+05
25%,19.250000,8.700000,1.450604e+06
50%,37.500000,9.000000,2.742122e+06
75%,55.750000,9.500000,4.125000e+06
max,74.000000,10.000000,1.734249e+07


In [ ]:
# Save data dalam csv untuk dilakukan pengolahan lanjutan dengan SQL
df.to_csv('hotel_bali_clean.csv', index=False)